In [2]:
# delete this cell if working on Pycharm
!pip install Bio
!pip install import-ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 270 kB 8.6 MB/s 
     |████████████████████████████████| 2.3 MB 68.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:

import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
import os
# so we can import utils notebook (delete if working on Pycharm), you might need to change it to your working directory path

from google.colab import drive 
drive.mount('/content/drive/')
!ls
%cd "drive"
%cd "MyDrive"
%cd "hackathonPrivate"
import import_ipynb
import utils


Mounted at /content/drive/
drive  sample_data
/content/drive
/content/drive/MyDrive
/content/drive/MyDrive/hackathonPrivate
importing Jupyter notebook from utils.ipynb
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
###############################################################################
#                                                                             #
#              Parameters you can change, but don't have to                   #
#                                                                             #
###############################################################################


# number of ResNet blocks for the first ResNet and the kernel size.
RESNET_1_BLOCKS = 3 
RESNET_1_KERNEL_SIZE = 15
RESNET_1_KERNEL_NUM = 64


###############################################################################
#                                                                             #
#                        Parameters you need to choose                        #
#                                                                             #
###############################################################################


# number of ResNet blocks for the second ResNet, dilation list to repeat and the kernel size.

RESNET_2_BLOCKS = 3
RESNET_2_KERNEL_SIZE = 3
RESNET_2_KERNEL_NUM = 32
DILATION = [1, 2, 4, 8]

# percentage of dropout for the dropout layer
DROPOUT = 0.15

# number of epochs, Learning rate and Batch size
EPOCHS = 60
LR = 0.01
BATCH = 32

In [5]:
def resnet_1(input_layer, res1_blocks, res1_ker_size, res1_ker_num, activation_res1):  # TODO: implement this!
    """
    ResNet layer - input -> BatchNormalization -> Conv1D -> Relu -> BatchNormalization -> Conv1D -> Relu -> Add
    :param input_layer: input layer for the ResNet
    :return: last layer of the ResNet
    """
    for _ in range(res1_blocks):
      out_batchnorm = layers.BatchNormalization()(input_layer)
      out_conv1 = layers.Conv1D(res1_ker_num, res1_ker_size, padding='same', activation=activation_res1)(out_batchnorm)
      out_batchnorm = layers.BatchNormalization()(out_conv1)
      out_conv1 = layers.Conv1D(res1_ker_num, res1_ker_size, padding='same', activation=activation_res1)(out_batchnorm)
      input_layer = layers.Add()([input_layer, out_conv1])
    return input_layer





In [6]:
def resnet_2(input_layer, dilaitons, res2_blocks, res2_ker_size, res2_ker_num, activation_res2):  # TODO: implement this!
    """
    Dilated ResNet layer - input -> BatchNormalization -> dilated Conv1D -> Relu -> BatchNormalization -> dilated Conv1D -> Relu -> Add
    :param input_layer: input layer for the ResNet
    :return: last layer of the ResNet
    """
    for _ in range(res2_blocks):
      for dilation in dilaitons:
        out_batchnorm = layers.BatchNormalization()(input_layer)
        out_conv1 = layers.Conv1D(res2_ker_num, res2_ker_size, padding='same', activation=activation_res2, dilation_rate=dilation)(out_batchnorm)
        out_batchnorm = layers.BatchNormalization()(out_conv1)
        out_conv1 = layers.Conv1D(res2_ker_num, res2_ker_size, padding='same', activation=activation_res2, dilation_rate=dilation)(out_batchnorm)
        input_layer = layers.Add()([input_layer, out_conv1])
    return input_layer

In [7]:
def build_network(res1_blocks = 3, res1_ker_size = 15, res1_ker_num = 64, activation_res1 = 'relu',
                  res2_blocks = 3, res2_ker_size = 3, res2_ker_num = 32, activation_res2 = 'relu',
                  dilaitons = [1, 2, 4, 8], dropout = 0.15, epoch = 60, 
                  batch =32, activation_dropout = 'elu'):
    """
    builds the neural network architecture as shown in the exercise.
    :return: a Keras Model
    """
    ...
    # input, shape (NB_MAX_LENGTH,FEATURE_NUM)
    input_layer = tf.keras.Input(shape=(utils.NB_MAX_LENGTH, utils.FEATURE_NUM))

    # Conv1D -> shape = (NB_MAX_LENGTH, RESNET_1_KERNEL_NUM)
    conv1d_layer = layers.Conv1D(res1_ker_num, res1_ker_size, padding='same')(input_layer)

    # first ResNet -> shape = (NB_MAX_LENGTH, RESNET_1_KERNEL_NUM)
    resnet_layer = resnet_1(conv1d_layer, res1_blocks, res1_ker_size, res1_ker_num, activation_res1)

    # # Conv1D -> shape = (NB_MAX_LENGTH, RESNET_2_KERNEL_NUM)
    conv1d_layer = layers.Conv1D(RESNET_2_KERNEL_NUM, RESNET_2_KERNEL_SIZE, padding="same")(resnet_layer)

    # # second ResNet -> shape = (NB_MAX_LENGTH, RESNET_2_KERNEL_NUM)
    resnet_layer = resnet_2(conv1d_layer, dilaitons, res2_blocks, res2_ker_size, res2_ker_num, activation_res2)

    dropout_layer = layers.Dropout(dropout)(resnet_layer)

    conv1d_layer = layers.Conv1D(np.ceil(res2_ker_num/2), res2_ker_size, padding="same", activation=activation_dropout)(dropout_layer)

    output = layers.Dense(utils.OUTPUT_SIZE)(conv1d_layer)


    return tf.keras.Model(input_layer, output, name='from_seq_to_struct')



In [8]:
def plot_val_train_loss(history):
    """
    plots the train and validation loss of the model at each epoch, saves it in 'model_loss_history.png'
    :param history: history object (output of fit function)
    :return: None
    """
    ig, axes = plt.subplots(1, 1, figsize=(15,3))
    axes.plot(history.history['loss'][10:], label='Training loss')
    axes.plot(history.history['val_loss'][10:], label='Validation loss')
    axes.legend()
    axes.set_title("Train and Val MSE loss")

    plt.savefig("/content/drive/MyDrive/hackathonPrivate/model_loss_history")  # TODO: you can change the path here


In [9]:
def train_models(num_models_to_train, meta_parametes):
    path_for_models =  "/content/drive/MyDrive/hackathonPrivate/models"
    X = np.load('/content/drive/MyDrive/hackathonPrivate/files/train_input.npy')
    Y = np.load('/content/drive/MyDrive/hackathonPrivate/files/train_labels.npy')

    X_train, X_val, y_train, y_val  = train_test_split(X, Y, test_size=0.2, random_state=1)
    for i in range(num_models_to_train):
        if (meta_parametes[i] == []):
          model = build_network()
          my_optimizer= tf.keras.optimizers.Adam(learning_rate=LR)
          model.compile(optimizer=my_optimizer, loss="mse")
          history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH, validation_data=(X_val, y_val))

          model.save("%s/our_model.tf" %path_for_models)
        else:
          pass


In [ ]:
if __name__ == '__main__':
    train_models(1, [[]])
    path_for_models
    model = tf.keras.models.load_model("%s/our_model.tf" %path_for_models)

    # part 3 predict
    to_predict = utils.generate_input(os.path.join("/content/drive/MyDrive/Hachathon Guy Yaniv Sagi Adi/Data/Ex4Data", "6xw6.pdb"))
    prediction = model.predict(to_predict[np.newaxis,:,:])
    matrix = utils.matrix_to_pdb(utils.get_seq_aa(os.path.join("/content/drive/hackathonPrivate", "6xw6.pdb"), "H")[0], prediction[0, :, :],"our_6xw6")


Epoch 1/60
